In [ ]:
%matplotlib qt 

#import bezier 
from scipy import interpolate
import pygame
import matplotlib.pyplot as plt
import numpy as np
from bicyclemodel import draw_car_indep  #x,y,yaw,v

def bezier_func(nodes,s_vals):
    eval = []
    for t in s_vals:
        evaluation = (1-t)**3*nodes[0] + 3*(1-t)**2*t*nodes[1]+3*(1-t)*t**2*nodes[2]+ t**3*nodes[3]

        eval.append(evaluation)
    eval = np.array(eval)
    #print(eval)
    return eval


def bezier_K(P,s_vals):
    eval = []
    for t in s_vals:
        B_d  = 3*(1-t)**2*(P[1]-P[0]) + 6*(1-t)*t*(P[2]-P[1]) + 3*t**2*(P[3]-P[2])
        
        B_dd = 6*(1-t)*(P[2]-2*P[1]-P[0]) + 6*t*(P[3]-2*P[2]-P[1])
        cross = np.cross(B_d,B_dd)
        absolute_B_d = np.absolute(B_d)
        absol = ((np.hypot(absolute_B_d[0],absolute_B_d[1]))**3)
        evaluation = cross/absol


        #print(f"{B_d},{B_dd},{cross},{absol} ")
        eval.append(evaluation)
    return np.array(eval)

def plot_spline(state1,state2,w1,w2,plot=None,plot_c_points=False):

    Point_1 = np.array([state1[0],state1[1]])
    Point_4 = np.array([state2[0],state2[1]])
    yaw1 = state1[2]
    yaw2 = state2[2]
    Point_2 = Point_1 + np.array([w1*np.cos(yaw1),w1*np.sin(yaw1)])
    Point_3 = Point_4 + np.array([-w2*np.cos(yaw2),-w2*np.sin(yaw2)])

    points = np.vstack((Point_1,Point_2,Point_3,Point_4))
    
    nodes = np.asfortranarray(points.T)
    #curve = bezier.Curve(nodes,degree=3)
    #curvepoints = curve.eval
    s_vals = np.linspace(0,1,300)
    #curve.plot(50,ax=plt)
    curve_points = bezier_func(points,s_vals)
    curvature = bezier_K(points,s_vals)
    max_curvature = np.amax(curvature)
    #print(curvature)

    #plt.plot(curve_points.T[0],curve_points.T[1])
    plot.scatter(curve_points.T[0],curve_points.T[1],s=5,c=curvature,cmap='jet',label = f"w1:{w1}, w2:{w2}, max K:{round(max_curvature,3)}")
    if plot_c_points:
        plot.plot(points.T[0],points.T[1],linestyle='dashed',color='black',)
        plot.scatter(points.T[0],points.T[1],color='black')

    return curve_points,max_curvature,points

state1 = np.array([0,0,0,80])
state2 = np.array([80,60,1.9,80])
plt.axis("equal")
draw_car_indep(state1,plot=plt,delta=0.0,drawvel=True)
draw_car_indep(state2,plot=plt,delta=0.0,drawvel=True)


#plot_spline(state1,state2,10,10,plot=plt)
weights = np.array([20,100])
for w1 in weights:
    for w2 in weights:

        curve_points , max_curvature,C_points = plot_spline(state1,state2,w1,w2,plot=plt)

        tck,u = interpolate.splprep([curve_points.T[0],curve_points.T[1]],s=0)
        unew = np.arange(0,1,0.01)
        out = interpolate.splev(unew,tck) 
        ind = int(np.array(out).shape[1]/2)
        x = out[0][ind]
        y = out[1][ind]
        plt.text(x,y,f"{round(max_curvature,3)}",fontsize=7,bbox=dict(boxstyle="round",
                   ec=(1., 0.5, 0.5),
                   fc=(1., 0.8, 0.8),
                   ))
        
plt.colorbar()
plt.title("Bezier Curve Testing")
plt.legend()

plt.show()


In [ ]:
_,_,C_points = plot_spline(state1,state2,40,30,plot=plt,plot_c_points=True)
plt.title("Bézier curve with visble control points")
plt.text(C_points[0][0],C_points[0][1]+3,"P1",fontsize=10)
plt.text(C_points[1][0],C_points[1][1]+3,"P2",fontsize=10)
plt.text(C_points[2][0],C_points[2][1]+3,"P3",fontsize=10)
plt.text(C_points[3][0]+3,C_points[3][1]-1,"P4",fontsize=10)
plt.show()